## Курсовой проект

**Основное**
- Дедлайн - 21 июня 23:59
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)  
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель - следите за валидацией 

In [259]:
!pip install implicit

Defaulting to user installation because normal site-packages is not writeable


In [260]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


### Подключение необходимых библиотек и модулей

In [261]:
import os
import inspect
import shutil

In [262]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from implicit import als

import lightgbm as lgb

from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
%matplotlib inline

import time
import pickle

import warnings
warnings.filterwarnings('ignore')

In [263]:
%matplotlib inline

In [264]:
warnings.filterwarnings('ignore')

In [265]:
# precision_at_k: вычисляет точность на первых k позициях.
# Использует функцию precision для оценки только первых k элементов из рекомендованного списка.
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [266]:
# recall_at_k: вычисляет полноту на первых k позициях. 
# Использует функцию recall для оценки только первых k элементов из рекомендованного списка.
def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)

In [280]:
'''
Функция prefilter_items выполняет предварительную фильтрацию данных с целью подготовки данных для рекомендаций.
Она выполняет следующие действия:

- Удаляет неинтересные для рекомендаций категории товаров на основе информации о категориях (item_features).
- Редкие категории, в которых количество уникальных товаров меньше 150, считаются неинтересными, и все товары из этих категорий удаляются из данных.
- Удаляет слишком дешевые товары, так как на них не заработаем.
-Товары, у которых стоимость (sales_value) деленная на количество продаж (quantity) меньше или равна 2, удаляются из данных.
- Удаляет слишком дорогие товары. Товары, у которых стоимость (sales_value) больше 50, удаляются из данных.

Оставляет только топ-N популярных товаров на основе суммарного количества продаж (quantity).
Топ-N товаров с наибольшим количеством продаж остаются в данных, остальные удаляются. Значение N задается параметром take_n_popular.
Заменяет item_id для всех товаров, которые не входят в топ-N популярных товаров, на фиктивный item_id 999999.
Это позволяет учесть факт покупки любого товара, который не попал в топ-N, как покупку фиктивного товара.
'''

def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем не интересные для рекоммендаций категории (department):
    if item_features is not None:
        department_size = pd.DataFrame(item_features. \
                                       groupby('department')['item_id'].nunique(). \
                                       sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[
            item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб:
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]
    
    # Уберем слишком дорогие товары:
    data = data[data['price'] < 50]

    # Возьмём топ по популярности:
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()


    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999

    return data

def postfilter_items(user_id, recommendations):
    pass

In [281]:
# создаём папку src в текущем рабочем каталоге и переносим туда функции,
# которые уже определены в текущем Jupyter Notebook,
# и выполним перенос функций в файл src/metrics.py.

# Создание папки src
src_dir = 'src'
os.makedirs(src_dir, exist_ok=True)

# Список функций для переноса
functions_to_move = [
    precision_at_k,
    recall_at_k,
    prefilter_items,
    postfilter_items
]

# Перенос функций в src.metrics.py
with open(os.path.join(src_dir, 'metrics.py'), 'a', encoding='utf-8') as f:
    for function in functions_to_move:
        function_source = inspect.getsource(function)
        f.write(function_source)
        f.write('\n')


In [282]:
# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data: pd.DataFrame):
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                        regularization=regularization,
                                        iterations=iterations,  
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
            
        return model

    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        
        # Получаем идентификатор пользователя из его имени
        user_id = self.userid_to_id[user]
        
        # Получаем список похожих пользователей
        similar_users = self.own_recommender.similar_users(user_id, N+1)
        
        # Удаляем исходного пользователя из списка похожих пользователей
        similar_users = similar_users[1:]
        
        # Получаем список товаров, купленных похожими пользователями
        items = []
        for similar_user_id in similar_users:
            recs = self.own_recommender.recommend(similar_user_id, self.user_item_matrix.T.tocsr(), N=1)
            items.append(self.id_to_itemid[recs[0][0]])
        
        # Удаляем дубликаты и возвращаем список рекомендаций
        res = list(set(items))
        
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    
    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # Получаем идентификатор пользователя из его имени
        user_id = self.userid_to_id[user]
        
        # Получаем топ-N товаров, купленных пользователем
        top_items = self.user_item_matrix.loc[user_id].sort_values(ascending=False).head(N)
        
        # Получаем похожие товары для каждого из топ-N товаров
        similar_items = []
        for item_id, score in top_items.iteritems():
            recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)
            similar_items.extend([self.id_to_itemid[rec[0]] for rec in recs[1:]])
        
        # Удаляем дубликаты и возвращаем список рекомендаций
        res = list(set(similar_items))
        
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

In [283]:
# Проверяем работу функций м классов
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

# from src.recommenders import MainRecommender 

*класс 'MainRecommender' не удается сохранить - возникла ошибка OSError: source code not available.  
Вручную вставленный код в файл src/recommenders.py тоже не хочет читаться.*

### Загрузка и проверка данных

In [284]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')
test = pd.read_csv('retail_test.csv')

In [285]:
data

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0


In [286]:
item_features

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ
...,...,...,...,...,...,...,...
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,


In [287]:
user_features

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16
...,...,...,...,...,...,...,...,...
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498


In [288]:
test

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
88729,98,41944918665,684,945779,2,2.00,421,0.0,1705,98,0.0,0.0
88730,98,41944918665,684,993617,2,2.00,421,0.0,1705,98,0.0,0.0
88731,98,41944918665,684,1128647,2,2.00,421,0.0,1705,98,0.0,0.0
88732,98,41944918665,684,9526886,2,0.60,421,0.0,1705,98,0.0,0.0


In [289]:
item_features.columns = [col.lower() for col in item_features.columns]
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)
user_features.columns = [col.lower() for col in user_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

### Деление данных на тренировочный, валидационный и тестовый датасеты

**Схема обучения:** -- давние покупки -- | -- 6 недель -- | -- 3 недели --

In [291]:
max_week = data['week_no'].max()
condition_train = (data['week_no'] < max_week - 9)
condition_valid = data['week_no'] >= max_week - 3
condition_test = ((data['week_no'] >= max_week - 9) & (data['week_no'] < max_week - 3))
                   
train_L1 = data[condition_train]
test_L1 = data[condition_test]

train_L2 = test_L1.copy()
valid_L2 = data[condition_valid]

### Предварительная фильтрация данных

In [292]:
n_items_before = train_L1['item_id'].nunique()
train_L1 = prefilter_items(train_L1, 
                           item_features=None, 
                           take_n_popular=10000-1)                    
n_items_after = train_L1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

NameError: name 'np' is not defined

### Генерация новых признаков для модели второго уровня

**Признаки пользователей**

Переводим категориальные признаки в числовые

In [293]:
user_features['age_desc'].replace(
    {'19-24': 22, '25-34': 30, '35-44': 40, '45-54': 50, '55-64': 60, '65+': 70},
    inplace=True)

user_features['marital_status_code'].replace(
    {'U': 0, 'A': 1, 'B': 2}, inplace=True)

user_features['income_desc'].replace(
    {'Under 15K': 10, '15-24K': 20, '25-34K':30, '35-49K': 40,
     '50-74K': 62, '75-99K': 87, '100-124K': 112, '125-149K': 137, 
     '150-174K': 162, '175-199K': 187, '200-249K': 225, '250K+':375}, inplace=True)

user_features['homeowner_desc'].replace(
    {'Unknown': 0, 'Probable Renter': 1, 'Renter': 2,
     'Probable Owner': 10, 'Homeowner': 20}, inplace=True)

user_features['hh_comp_desc'].replace(
    {'Unknown': 0, 'Single Male': 1, 'Single Female': 2,
     '1 Adult Kids': 4, '2 Adults No Kids': 5, '2 Adults Kids':6},inplace=True)

user_features['household_size_desc'].replace({'5+': 6}, inplace=True) 

user_features['kid_category_desc'].replace(
    {'None/Unknown': 0, '3+': 5}, inplace=True)

user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,70,1,40,20,5,2,0,1
1,50,1,62,20,5,2,0,7
2,30,0,30,0,6,3,1,8
3,30,0,87,20,6,4,2,13
4,50,2,62,20,2,1,0,16


**Признаки товаров**

In [294]:
item_features['department'] = item_features['department'].astype('category')

counts = item_features['department'].value_counts()
others = counts[counts < 50].index

label = 'OTHER'

item_features['department'] = item_features['department'].cat.add_categories([label])
item_features['department'] = item_features['department'].replace(others, label)

item_features['commodity_desc'] = item_features['commodity_desc'].astype('category')

counts = item_features['commodity_desc'].value_counts()
others = counts[counts < 50].index

label = 'OTHER'

item_features['commodity_desc'] = item_features['commodity_desc'].cat.add_categories([label])
item_features['commodity_desc'] = item_features['commodity_desc'].replace(others, label)

commodities = item_features.commodity_desc.value_counts()
commodities_list = commodities.keys().tolist()
for i, name in enumerate(commodities_list):
    item_features.loc[item_features['commodity_desc'] == name, 'commodity_category'] = i
    
item_features['brand'] = np.where(item_features['brand']=='Private', 0, 1)

item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,commodity_category
0,25671,2,GROCERY,1,OTHER,ICE - CRUSHED/CUBED,22 LB,12.0
1,26081,2,MISC. TRANS.,1,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,57.0
2,26093,69,PASTRY,0,BREAD,BREAD:ITALIAN/FRENCH,,92.0
3,26190,69,GROCERY,0,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ,74.0
4,26355,69,GROCERY,0,COOKIES/CONES,SPECIALTY COOKIES,14 OZ,17.0


**Признаки товаров пользователей и товаров**

In [295]:
def get_user_item_features(data_train_lvl_1):
    # час совершения транзакции
    X = data_train_lvl_1.copy()
    X['hour'] = X['trans_time'] // 100
    user_item_features = X.groupby(['user_id', 'item_id'])['hour'].median().reset_index()
    user_item_features.columns = ['user_id', 'item_id', 'median_sales_hour']
    
    # день недели совершения транзакции
    X['weekday'] = X['day'] % 7
    df = X.groupby(['user_id', 'item_id'])['weekday'].median().reset_index()
    df.columns = ['user_id', 'item_id', 'median_weekday']
    user_item_features = user_item_features.merge(df, on=['user_id', 'item_id'])
    
    # средний чек корзины клиента
    df = X.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
    df = df.groupby('user_id')['sales_value'].mean().reset_index()
    df.columns = ['user_id', 'mean_check']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # количество магазинов, в которых продавался товар
    df = X.groupby(['item_id'])['store_id'].nunique().reset_index()
    df.columns = ['item_id', 'n_stores']
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # количество уникальных товаров, купленных клиентом
    df = X.groupby(['user_id'])['item_id'].nunique().reset_index()
    df.columns = ['user_id', 'n_items']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # количество транзакций клиента
    df = X.groupby(['user_id'])['item_id'].count().reset_index()
    df.columns = ['user_id', 'n_transactions']
    user_item_features = user_item_features.merge(df, on=['user_id'])

    # эмбеддинги товаров
    recommender = MainRecommender(X)
    df = recommender.model.item_factors
    n_factors = recommender.model.factors
    ind = list(recommender.id_to_itemid.values())
    df = pd.DataFrame(df, index=ind).reset_index()
    df.columns = ['item_id'] + ['item_f_' + str(i + 1) for i in range(n_factors)]
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # эмбеддинги пользователей
    df = recommender.model.user_factors
    ind = list(recommender.id_to_userid.values())
    df = pd.DataFrame(df, index=ind).reset_index()
    df.columns = ['user_id'] + ['user_f_' + str(i + 1) for i in range(n_factors)]
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    return user_item_features

In [ ]:
%%time

user_item_features = get_user_item_features(train_L1)
user_item_features.head()

### Подготовка данных для модели второго уровня

In [297]:
def get_candidates(data_train_lvl_1, 
                   data_train_lvl_2, 
                   N, 
                   add_to_lvl_2):
    recommender = MainRecommender(data_train_lvl_1)

    users_lvl_1 = data_train_lvl_1['user_id'].unique()
    users_lvl_2 = data_train_lvl_2['user_id'].unique().tolist()
    if add_to_lvl_2:
        users_lvl_2 += add_to_lvl_2

    current_users = list(set(users_lvl_2) & set(users_lvl_1))    
    new_users = list(set(users_lvl_2) - set(users_lvl_1))

    df = pd.DataFrame(users_lvl_2, columns=['user_id'])
    cond_1 = df['user_id'].isin(current_users)
    df.loc[cond_1, 'candidates'] = df.loc[cond_1, 'user_id'].apply(
        lambda x: recommender.get_own_recommendations(x, N))

    if new_users:
        cond_2 = df['user_id'].isin(new_users)
        df.loc[cond_2, 'candidates'] = df.loc[cond_2, 'user_id'].apply(
            lambda x: recommender.overall_top_purchases[:N])
        
    return df

In [298]:
def get_targets_lvl_2(data_train_lvl_1, 
                      data_train_lvl_2, 
                      user_item_features, 
                      N, 
                      add_to_lvl_2=None):
    
    users_lvl_2 = get_candidates(data_train_lvl_1, 
                                 data_train_lvl_2, 
                                 N, 
                                 add_to_lvl_2)
    
    df = pd.DataFrame({'user_id': users_lvl_2['user_id'].values.repeat(N),
                       'item_id': np.concatenate(users_lvl_2['candidates'].values)})

    targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
    targets_lvl_2['target'] = 1 

    targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
    targets_lvl_2['target'].fillna(0, inplace= True)
    
    targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(
        user_item_features, on=['user_id', 'item_id'], how='left')
    
    return targets_lvl_2

In [299]:
users_train_L1 = train_L1.user_id.unique()
users_test_L1 = test_L1.user_id.unique()
users_valid_L2 = valid_L2.user_id.unique()

new_users_test_to_train = list(set(users_test_L1) - set(users_train_L1))

new_users_L1_to_valid = list(set(users_valid_L2) - (set(users_train_L1) | set(users_test_L1)))

add_to_lvl_2 = list(set(users_valid_L2) - (set(users_test_L1)))

new_users_test_to_train, new_users_L1_to_valid, len(add_to_lvl_2)

([1984], [], 126)

In [ ]:
%%time

N = 1000

targets_lvl_2 = get_targets_lvl_2(train_L1, 
                                  train_L2, 
                                  user_item_features, 
                                  N, 
                                  add_to_lvl_2)

print(f'Число пользователей: {targets_lvl_2.user_id.nunique()}')

targets_lvl_2.head()

### Отбор признаков

In [301]:
SELECTED_FEATURES_NAMES = ['brand',
                           'commodity_category',
                           'age_desc', 
                           'income_desc', 
                           'homeowner_desc', 
                           'hh_comp_desc',
                           'marital_status_code', 
                           'manufacturer',                            
                           'median_sales_hour', 
                           'median_weekday', 
                           'mean_check', 
                           'n_stores', 
                           'n_items', 
                           'n_transactions', 
                           'item_f_1', 
                           'item_f_2', 
                           'item_f_3', 
                           'item_f_4', 
                           'item_f_5',
                           'user_f_1', 
                           'user_f_2', 
                           'user_f_3', 
                           'user_f_4',
                           'user_f_5',
                          ]
categorical = ['marital_status_code',
               'homeowner_desc', 
               'hh_comp_desc', 
               'manufacturer',
               'commodity_category'
              ]

### Подготовка моделей

In [302]:
def run_model_lgb(targets_lvl_2):
    X_train, X_valid, y_train, y_valid = train_test_split(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0),
                                                          targets_lvl_2[['target']],
                                                          test_size=0.2, random_state=2021,
                                                          stratify=targets_lvl_2[['target']])

    dtrain = lgb.Dataset(X_train, y_train, categorical_feature=categorical)
    dvalid = lgb.Dataset(X_valid, y_valid, categorical_feature=categorical)

    params_lgb = {"boosting_type": "gbdt",
                  "objective": "binary", 
                  "metric": "auc",
                  "num_boost_round": 10000,
                  "learning_rate": 0.1,
                  "num_leaves": 30,
                  "max_depth": 10,
                  "n_estimators": 5000,
                  "n_jobs": 6,
                  "seed": 2021} 

    model_lgb = lgb.train(params=params_lgb,
                          train_set=dtrain,  
                          valid_sets=[dtrain, dvalid],
                          categorical_feature=categorical,
                          verbose_eval=1000,
                          early_stopping_rounds=10)
    
    return model_lgb

In [ ]:
%%time

model_lgb = run_model_lgb(targets_lvl_2)

In [304]:
SELECTED_FEATURES_NAMES_cb = [i for i in SELECTED_FEATURES_NAMES if not i in categorical]

def run_model_cb(targets_lvl_2):    
    X_train, X_valid, y_train, y_valid = train_test_split(targets_lvl_2[SELECTED_FEATURES_NAMES_cb].fillna(0),
                                                          targets_lvl_2[['target']],
                                                          test_size=0.2, random_state=2021,
                                                          stratify=targets_lvl_2[['target']])

    dtrain = Pool(data=X_train, label=y_train)
    dvalid = Pool(data=X_valid, label=y_valid)

    params_cb = {"n_estimators":5000,
                 "learning_rate": 0.1,
                 "loss_function": "Logloss",
                 "eval_metric": "AUC",
                 "task_type": "CPU",
                 "max_bin": 30,
                 "early_stopping_rounds": 30,
                 "verbose": 100,
                 "max_depth": 10,
                 "l2_leaf_reg": 80,
                 "thread_count": 6,
                 "random_seed": 2021,
                 } 

    model_cb = CatBoostClassifier(**params_cb)
    
    model_cb.fit(dtrain, eval_set=[dvalid],verbose_eval=False)

    return model_cb

In [ ]:
%%time

model_cb = run_model_cb(targets_lvl_2)

In [ ]:
predictions_lgb_train = model_lgb.predict(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0))
predictions_cb_train = model_cb.predict_proba(targets_lvl_2[SELECTED_FEATURES_NAMES_cb].fillna(0))[:, 1]

### Объединение прогнозов двух моделей

In [ ]:
preds_train = pd.DataFrame(zip(predictions_lgb_train, 
                               predictions_cb_train),
                           columns=['lgb', 'cb']).mean(axis=1).values
roc_auc_score(targets_lvl_2['target'], preds_train)

### Обработка прогнозов

In [ ]:
def get_predictions(targets_lvl_2, raw_predictions, prefix='lgb'): 
    df = targets_lvl_2[['user_id', 'item_id']]
    df['predictions'] = raw_predictions

    df = df.groupby(['user_id', 'item_id'])['predictions'].median().reset_index()
    df = df.sort_values(['predictions'], ascending=False).groupby(['user_id']).head(5)

    df = df.groupby('user_id')['item_id'].unique().reset_index()
    df.columns = ['user_id', prefix + '_recommendations']
    
    return df

def get_results(data_val_lvl_2, targets_lvl_2, preds_lgb, preds_cb, combined_preds):
    result = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
    result.columns=['user_id', 'actual']
    
    prefixes = ['lgb', 'cb', 'cb_lgb']
    predictions = [preds_lgb, preds_cb, combined_preds]
    
    for i, preds in enumerate(predictions):
        df = get_predictions(targets_lvl_2, preds, prefixes[i])
        result = result.merge(df, on='user_id', how='left')

    return result

In [ ]:
result_lvl_2 = get_results(valid_L2, 
                           targets_lvl_2, 
                           predictions_lgb_train,
                           predictions_cb_train, 
                           preds_train)
result_lvl_2.head()

### Расчет метрик

#### LightGBM

In [ ]:
result_lvl_2.apply(lambda row: precision_at_k(row['lgb_recommendations'], 
                                              row['actual'], 
                                              5), axis=1).mean()

#### CatBoost

In [ ]:
result_lvl_2.apply(lambda row: precision_at_k(row['cb_recommendations'], 
                                              row['actual'], 
                                              5), axis=1).mean()

#### Ансамбль моделей

In [ ]:
result_lvl_2.apply(lambda row: precision_at_k(row['cb_lgb_recommendations'], 
                                              row['actual'], 
                                              5), axis=1).mean()

### Построение финальных прогнозов

In [ ]:
validation_weeks = 6
data_train = data[data['week_no'] < data['week_no'].max() - validation_weeks]
data_valid = data[data['week_no'] >= data['week_no'].max() - validation_weeks]

In [ ]:
users_lvl_1 = data_train.user_id.unique()
users_lvl_2 = data_valid.user_id.unique()
users_lvl_3 = test.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

new_users_lvl_2, new_users_lvl_3, len(add_to_lvl_2)

In [ ]:
n_items_before = data['item_id'].nunique()
data_train = prefilter_items(data_train, 
                           item_features=None, 
                           take_n_popular=10000-1)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

In [ ]:
user_item_features = get_user_item_features(data_train)
user_item_features.head()

In [ ]:
targets_test = get_targets_lvl_2(data_train, 
                                 data_valid, 
                                 user_item_features, 
                                 N, 
                                 add_to_lvl_2)

print(f'Число пользователей: {targets_test.user_id.nunique()}')
print(f'Среднее число покупок: {round(targets_test["target"].mean(), 4)}')

targets_test.head()

#### Генерация прогнозов

In [ ]:
model_cb = run_model_cb(targets_test)

In [ ]:
predictions_lgb_test = model_lgb.predict(targets_test[SELECTED_FEATURES_NAMES].fillna(0))
predictions_cb_test = model_cb.predict_proba(targets_test[SELECTED_FEATURES_NAMES_cb].fillna(0))[:, 1]

In [ ]:
preds_test = pd.DataFrame(zip(predictions_lgb_test, predictions_cb_test),
                          columns=['lgb', 'cb']).mean(axis=1).values
roc_auc_score(targets_test['target'], preds_test)

In [ ]:
result_test = get_results(test, targets_test, 
                           predictions_lgb_test,
                           predictions_cb_test, 
                           preds_test)
result_test.head()

In [ ]:
result_test.apply(lambda row: precision_at_k(row['lgb_recommendations'],
                                             row['actual'],
                                             5), axis=1).mean()

In [ ]:
result_test.apply(lambda row: precision_at_k(row['cb_recommendations'], 
                                             row['actual'], 
                                             5), axis=1).mean()

In [ ]:
result_test.apply(lambda row: precision_at_k(row['cb_lgb_recommendations'], 
                                             row['actual'], 
                                             5), axis=1).mean()

### Запись результата в файл

In [ ]:
df = result_test[['user_id', 'cb_lgb_recommendations']].copy()
df.to_csv('recommendations.csv', index=False)

df